In [40]:
import json
import os
import re

import langid
import pandas as pd
from pandas_profiling import ProfileReport
import pymorphy2
import tokenize_uk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [2]:
langid.set_languages(['ru','uk'])
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [3]:
data = {}
data_dirname = '/Users/dmytrobudashnyi/usr/projects/mine/twitter_scraper/rozetka_goods.json_tmp/'
for filename in os.listdir(data_dirname):
    with open(data_dirname + filename) as f:
        data.update(json.load(f))

len(data)

136047

In [4]:
data['0ef244fbcb9c7acb421dbf03a8f4e2e4']

{'user_id': 'Ваня Цитрус',
 'rating': 'Рейтинг товара 5 из 5',
 'content': 'Суперр розетки спасибо Недоліки: Нет'}

In [5]:
rating_pattern = re.compile(r"^Рейтинг товара (\d) из 5$")


def get_rating_digit(comment):
    groups = rating_pattern.search(comment['rating']).groups()
    if groups:
        return groups[0]

def get_lang(comment):
    return langid.classify(comment['content'].replace('Недоліки:', '').replace('Переваги:', ''))[0]

uk_comments = {}
for key, comment in data.items():
    lang = get_lang(comment)
    if lang == 'uk':
        rating = get_rating_digit(comment)
        if rating in ('1', '2', '3', '4', '5'):
            digit_rating = int(rating)
            text = comment['content'].strip()
            uk_comments[key] = {
                'user_id': comment['user_id'],
                'text': text,
                'rating': digit_rating,
                '_ends_with_?': text.endswith('?'),
            }

In [9]:
len(uk_comments)/len(data), len(uk_comments)

(0.15959190573845802, 21712)

![16%](./ua_16.png)

In [10]:
for key in list(uk_comments.keys())[:20]:
    print(uk_comments[key])

{'user_id': 'Влад.', 'text': "Топова пам'ять за свої гроші класно виглядає в корпусі Переваги: Частота скорость дезайн наклейка карл 60 місяців гарантії це однозначно великий плюс Недоліки: Ніяких!!!)", 'rating': 5, '_ends_with_?': False}
{'user_id': 'Денис Єгоров', 'text': 'Поставив дві планки у MSI Tomahawk Max - у біосі просто обрав частоту 3200 і все працює без проблем. Я задоволений, що сказати)', 'rating': 5, '_ends_with_?': False}
{'user_id': 'Максим Скориков', 'text': 'модем на перший погляд непоганий. При швидкості по кабелю 85Мб/с по вай фай роздає від 45 до 50Мб/с., що доволі непогано порівняно з іншими, які ріжуть швидкість втричі. Налаштування важкі - відрізняються від інших роутерів. В службі підтримки киевстар сидять жахливі консультанти, які навіть не знають яка в них МАС адреса та який вони використовують протокол. При швидкому налаштуванні підключити роутер не вийде - треба переходити в ручне керування і головне це виставити МАС адресу за замовчуванням та протокол:"не

In [11]:
all_comments_df = pd.DataFrame(uk_comments).T
all_comments_df

,user_id,text,rating,_ends_with_?
20029cc45769724050352ed544f396c7,Влад.,Топова пам'ять за свої гроші класно виглядає в...,5,False
ae7f8bafe762b6d64a7ddf2071991d6d,Денис Єгоров,Поставив дві планки у MSI Tomahawk Max - у біо...,5,False
1de61284cc3bc7224b857e723c1bec7a,Максим Скориков,модем на перший погляд непоганий. При швидкост...,4,False
145d013d3b7252487964796f8652ebeb,Роман Гавриленко,Дуже чітке зображення та хороша передача кольо...,5,False
2a1a4963ab1e9b71534b15e8436257b2,Іван Бірковський,А можно до монітора подключіть комп'ютер? Пере...,5,False
...,...,...,...,...
297bce6ec938accdc5ea6772df789814,Олег Великий,Які ігри на ньму підуть і наяких настройках,5,False
fad6a5be690a1b3f99595babf6fb5110,Денис Гордиенко,Можно установить видеокарту Asus 1050 Ti ROG?,5,True
007e0638ff0284c80e684351a30e06fb,Matrix TV,Якщо я підключу ПК до vga-монітора картинка бу...,5,False
13fb2ab8b08798ca9f3e3ef088a36963,Max Bro,Чи можна доставити операційній. Системі на 16,4,False


In [13]:
#cleaned_comments_df = all_comments_df[all_comments_df['_ends_with_?'] == False].drop(['_ends_with_?'], axis=1)
skip_indexes = all_comments_df[all_comments_df['_ends_with_?']][all_comments_df['rating'] == 5].index
cleaned_comments_df = all_comments_df.drop(skip_indexes, axis=0)
cleaned_comments_df.drop(['_ends_with_?'], axis=1, inplace=True)
cleaned_comments_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,user_id,text,rating
20029cc45769724050352ed544f396c7,Влад.,Топова пам'ять за свої гроші класно виглядає в...,5
ae7f8bafe762b6d64a7ddf2071991d6d,Денис Єгоров,Поставив дві планки у MSI Tomahawk Max - у біо...,5
1de61284cc3bc7224b857e723c1bec7a,Максим Скориков,модем на перший погляд непоганий. При швидкост...,4
145d013d3b7252487964796f8652ebeb,Роман Гавриленко,Дуже чітке зображення та хороша передача кольо...,5
2a1a4963ab1e9b71534b15e8436257b2,Іван Бірковський,А можно до монітора подключіть комп'ютер? Пере...,5
...,...,...,...
9e55df94ad5c6568eb5fc307efb13496,Віталій,Прийшов досить таки швидко. Відправили в день ...,5
297bce6ec938accdc5ea6772df789814,Олег Великий,Які ігри на ньму підуть і наяких настройках,5
007e0638ff0284c80e684351a30e06fb,Matrix TV,Якщо я підключу ПК до vga-монітора картинка бу...,5
13fb2ab8b08798ca9f3e3ef088a36963,Max Bro,Чи можна доставити операційній. Системі на 16,4


In [14]:
for index, row in cleaned_comments_df.iterrows():
    text = row['text']
    sentences = tokenize_uk.tokenize_sents(text)
    lemma_sentences = []
    for sentence in sentences:
        tokens = tokenize_uk.tokenize_words(sentence)
        lemma_sentences.append(' '.join([morph.parse(word)[0].normal_form for word in tokens]))

    if row['rating'] == 5:
        new_rating = 'positive'
    elif row['rating'] == 4:
        new_rating = 'neutral'
    else:
        new_rating = 'negative'

    cleaned_comments_df.loc[index, 'new_rating'] = new_rating
    cleaned_comments_df.loc[index, 'lemmatized_text'] = ' '.join(lemma_sentences)

In [15]:
cleaned_comments_df

,user_id,text,rating,new_rating,lemmatized_text
20029cc45769724050352ed544f396c7,Влад.,Топова пам'ять за свої гроші класно виглядає в...,5,positive,топовий пам'ять за свій гріш класно виглядати ...
ae7f8bafe762b6d64a7ddf2071991d6d,Денис Єгоров,Поставив дві планки у MSI Tomahawk Max - у біо...,5,positive,поставити два планка у msi tomahawk max - у бі...
1de61284cc3bc7224b857e723c1bec7a,Максим Скориков,модем на перший погляд непоганий. При швидкост...,4,neutral,модем на перший погляд непоганий . перти швидк...
145d013d3b7252487964796f8652ebeb,Роман Гавриленко,Дуже чітке зображення та хороша передача кольо...,5,positive,дуже чіткий зображення та хороший передача кол...
2a1a4963ab1e9b71534b15e8436257b2,Іван Бірковський,А можно до монітора подключіть комп'ютер? Пере...,5,positive,а можно до монітор подключити комп'ютер ? пере...
...,...,...,...,...,...
9e55df94ad5c6568eb5fc307efb13496,Віталій,Прийшов досить таки швидко. Відправили в день ...,5,positive,прийти досить таки швидко . відправити в день ...
297bce6ec938accdc5ea6772df789814,Олег Великий,Які ігри на ньму підуть і наяких настройках,5,positive,який гра на ньм піти і наякий настройка
007e0638ff0284c80e684351a30e06fb,Matrix TV,Якщо я підключу ПК до vga-монітора картинка бу...,5,positive,якщо я підключити пк до vga - монітор картинка...
13fb2ab8b08798ca9f3e3ef088a36963,Max Bro,Чи можна доставити операційній. Системі на 16,4,neutral,чи можна доставити операційний . система на 16


In [16]:
cleaned_comments_df.groupby(['new_rating']).count() / len(cleaned_comments_df) * 100

,user_id,text,rating,lemmatized_text
new_rating,,,,
negative,12.940039,12.940039,12.940039,12.940039
neutral,17.577369,17.577369,17.577369,17.577369
positive,69.482592,69.482592,69.482592,69.482592


In [17]:
df = cleaned_comments_df.copy(deep=True)
y = df['new_rating']
df.drop(['new_rating'], axis=1, inplace=True)
df

,user_id,text,rating,lemmatized_text
20029cc45769724050352ed544f396c7,Влад.,Топова пам'ять за свої гроші класно виглядає в...,5,топовий пам'ять за свій гріш класно виглядати ...
ae7f8bafe762b6d64a7ddf2071991d6d,Денис Єгоров,Поставив дві планки у MSI Tomahawk Max - у біо...,5,поставити два планка у msi tomahawk max - у бі...
1de61284cc3bc7224b857e723c1bec7a,Максим Скориков,модем на перший погляд непоганий. При швидкост...,4,модем на перший погляд непоганий . перти швидк...
145d013d3b7252487964796f8652ebeb,Роман Гавриленко,Дуже чітке зображення та хороша передача кольо...,5,дуже чіткий зображення та хороший передача кол...
2a1a4963ab1e9b71534b15e8436257b2,Іван Бірковський,А можно до монітора подключіть комп'ютер? Пере...,5,а можно до монітор подключити комп'ютер ? пере...
...,...,...,...,...
9e55df94ad5c6568eb5fc307efb13496,Віталій,Прийшов досить таки швидко. Відправили в день ...,5,прийти досить таки швидко . відправити в день ...
297bce6ec938accdc5ea6772df789814,Олег Великий,Які ігри на ньму підуть і наяких настройках,5,який гра на ньм піти і наякий настройка
007e0638ff0284c80e684351a30e06fb,Matrix TV,Якщо я підключу ПК до vga-монітора картинка бу...,5,якщо я підключити пк до vga - монітор картинка...
13fb2ab8b08798ca9f3e3ef088a36963,Max Bro,Чи можна доставити операційній. Системі на 16,4,чи можна доставити операційний . система на 16


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df, y, train_size=0.8, test_size=0.2, random_state=0, shuffle=True, stratify=y
)
train_data = X_train.join(y_train)
test_data = X_test.join(y_test)
train_data

,user_id,text,rating,lemmatized_text,new_rating
0c811a8fc0497f8a6de3f254dc24e5a8,Богдан Городенський,rozetka прислали жорсткий диск в куску картонк...,1,rozetka прислати жорсткий диск в кусок картонк...,negative
4870dd047a08460d0240ccdcb89e58e0,Олег Логвиненк,"Використовую вже 7 місяців, все влаштовує. Пер...",5,"використовувати вже 7 місяць , весь влаштовува...",positive
005d6ac9a642474c1162c9671e6bae99,Тарас,"Досвід користування невеликий, та перше вражен...",4,"досвід користування невеликий , та перший враж...",neutral
ab08664888c4105e2d3a56da076b8a01,Роман Гевко,дуже тихий вентилятор,4,дуже тихий вентилятор,neutral
652eb8f8e00837a9913726e55e17c07c,Юрій Колодій,"Клавіатура просто супер, ставлю 10/10 рекоменд...",5,"клавіатура просто супер , ставити 10 / 10 реко...",positive
...,...,...,...,...,...
27304de29a0701607b6c3cdce449c332,Сергій,"Память як память, працю і добре!",5,"память як память , праця і добре !",positive
cd3bca92571e792eaab13f24038534a5,Володимир Туз,Купував менше року тому. Спочатку - наче все н...,1,купувати менше рок том . спочатку - наче весь ...,negative
1885029ccc9cadcbb9f8da11962787bc,Ксе,брала на подарок мужу. Ему очень понравились. ...,5,брати на подарка муж . ему очення понравитися ...,positive
5dbec104c03d1011356c62d6635f810c,Godrik,"Гарно виглядає, чітко працює. IPS матриця роби...",5,"гарно виглядати , чітко працювати . ips матриц...",positive


In [19]:
train_profile = ProfileReport(train_data)
train_profile.to_file('train_profile_report.html')

/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/base.py:319: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [20]:
test_profile = ProfileReport(test_data)
test_profile.to_file('test_profile_report.html')

/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/base.py:319: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


# Baseline FTW

In [21]:
baseline = ['positive'] * len(test_data)
print(classification_report(test_data.new_rating.to_list(), baseline))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       535
     neutral       0.00      0.00      0.00       727
    positive       0.69      1.00      0.82      2874

    accuracy                           0.69      4136
   macro avg       0.23      0.33      0.27      4136
weighted avg       0.48      0.69      0.57      4136



/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Tokens + CountVectorizer + MultinomialNB

In [52]:
count_vect = CountVectorizer()

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, train_data.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

train_counts = count_vect.fit_transform(train_data.text)
print(train_counts.shape)
count_vect.get_feature_names()[5000:5020]

[cross_val] F1: 0.526215580296729
(16544, 46933)


['rainbow',
 'rainmetter',
 'rally',
 'ram',
 'ramaxel',
 'ramdisk',
 'random',
 'rapid',
 'rapoo',
 'rapoo_3100p_black',
 'rappo',
 'rar',
 'ras',
 'rasman',
 'rast',
 'rate',
 'ratio',
 'raw',
 'ray',
 'rayzen']

In [53]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(train_counts, train_data.new_rating.to_list())

In [54]:
test_counts = count_vect.transform(test_data.text)
y_pred = clf.predict(test_counts)

In [55]:
from sklearn.metrics import classification_report
print(classification_report(test_data.new_rating.to_list(), y_pred))

              precision    recall  f1-score   support

    negative       0.76      0.38      0.51       535
     neutral       0.42      0.07      0.12       727
    positive       0.75      0.98      0.85      2874

    accuracy                           0.74      4136
   macro avg       0.64      0.48      0.49      4136
weighted avg       0.69      0.74      0.68      4136



# Lemma + CountVectorizer + Multinomial NB

In [56]:
count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train_data.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, train_data.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, train_data.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (16544, 27656)
[cross_val] F1: 0.5298869072370327
Test counts shape: (4136, 27656)
              precision    recall  f1-score   support

    negative       0.72      0.46      0.56       535
     neutral       0.39      0.11      0.17       727
    positive       0.77      0.96      0.86      2874

    accuracy                           0.75      4136
   macro avg       0.62      0.51      0.53      4136
weighted avg       0.70      0.75      0.70      4136



# Undersampling

In [27]:
negative_df = train_data.loc[train_data['new_rating'] == 'negative']
n = len(negative_df)
print('Predicted number of records:', n * 3)
neutral_df = train_data.loc[train_data['new_rating'] == 'neutral'].sample(n=n,random_state=0)
positive_df = train_data.loc[train_data['new_rating'] == 'positive'].sample(n=n,random_state=0)
normalized_df = pd.concat([negative_df, neutral_df, positive_df])
normalized_df

Predicted number of records: 6423


,user_id,text,rating,lemmatized_text,new_rating
0c811a8fc0497f8a6de3f254dc24e5a8,Богдан Городенський,rozetka прислали жорсткий диск в куску картонк...,1,rozetka прислати жорсткий диск в кусок картонк...,negative
e189a52c092accb7ec4d157ef72e8d57,Юра,Ноутбук не загружається з цією оперативкою. Ск...,1,ноутбук не загружаватися з цей оперативка . ск...,negative
484885177a0825a7ad30791461fefd52,Сергей Олыва,"Загалом нормальний дешевий роутер, але сигнал ...",3,"загал нормальний дешевий роутерти , але сигнал...",negative
77bde83134cf22bf5c7e308ed2910e4a,Грицько Тара,Абсолютно не рекомендую Недоліки: якість жахли...,1,абсолютно не рекомендувати недолік : якість жа...,negative
b746cfae94bcaa22d35e4bb42f83dc32,Елена Алиманска,"на роботі така сама мишка і працює відмінно,мя...",1,на робот такий самий мишка і працювати відмінн...,negative
...,...,...,...,...,...
5bb65ced39000189c2ec14a73124ff00,Павел Осоки,"2 місяці ще не показник, але взагалом всім зад...",5,"2 місяць ще не показник , але взагал весь задо...",positive
413091665d0922ed2ead7722670c738a,Микола покупець,Поки все норм! Переваги: Дизайн Недоліки: Несу...,5,поки весь норма ! перевага : дизайн недолік : ...,positive
74306170ae1cb107d463111592be25da,Gotarus,"Чудова ""оперативка"" встановив у ноутбук близьк...",5,"чудовий "" оперативка "" встановити у ноутбук бл...",positive
dacd0f23d4755ac4d2bc31065fad131f,Анатолій Дощанський,"Купував для 3-4gмодема(Н.е3372-607),сімпара ві...",5,"купувати для 3-4 gмодем ( н . е3372 -607 ) , с...",positive


In [58]:
count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(normalized_df.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, normalized_df.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, normalized_df.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (26642, 27656)
[cross_val] F1: 0.7546246212892258
Test counts shape: (4136, 27656)
              precision    recall  f1-score   support

    negative       0.59      0.58      0.59       535
     neutral       0.34      0.35      0.34       727
    positive       0.84      0.83      0.83      2874

    accuracy                           0.71      4136
   macro avg       0.59      0.59      0.59      4136
weighted avg       0.72      0.71      0.71      4136



-----------------------
Щось пішло не так :|
Очікування були вищими

# Bigrams

In [61]:
count_vect = CountVectorizer(ngram_range=(2,2))
train_counts = count_vect.fit_transform(train_data.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, train_data.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, train_data.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (16544, 253395)
[cross_val] F1: 0.5633846972406789
Test counts shape: (4136, 253395)
              precision    recall  f1-score   support

    negative       0.84      0.33      0.48       535
     neutral       0.39      0.04      0.07       727
    positive       0.74      0.99      0.85      2874

    accuracy                           0.74      4136
   macro avg       0.65      0.45      0.46      4136
weighted avg       0.69      0.74      0.66      4136



In [62]:
count_vect = CountVectorizer(ngram_range=(1,2))
train_counts = count_vect.fit_transform(train_data.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, train_data.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, train_data.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (16544, 281051)
[cross_val] F1: 0.4827222225741963
Test counts shape: (4136, 281051)
              precision    recall  f1-score   support

    negative       0.85      0.22      0.35       535
     neutral       0.47      0.01      0.02       727
    positive       0.72      1.00      0.84      2874

    accuracy                           0.72      4136
   macro avg       0.68      0.41      0.40      4136
weighted avg       0.69      0.72      0.63      4136



# Stopwords

In [63]:
stopwords = [
    'авжеж', 'адже', 'але', 'б', 'без', 'був', 'була', 'були', 'було', 'бути', 'більш', 'вам', 'вас', 'весь', 'вздовж', 'ви',
    'вниз', 'внизу', 'вона', 'вони', 'воно', 'все', 'всередині', 'всіх', 'від', 'він', 'да', 'давай', 'давати', 'де', 'дещо',
    'для', 'до', 'з', 'завжди', 'замість', 'й', 'коли', 'ледве', 'майже', 'ми', 'навколо', 'навіть', 'нам', 'от', 'отже', 'отож',
    'поза', 'про', 'під', 'та', 'так', 'такий', 'також', 'те', 'ти', 'тобто', 'тож', 'тощо', 'хоча', 'це', 'цей', 'чи', 'чого',
    'що', 'як', 'який', 'якої', 'є', 'із', 'інших', 'їх', 'її', 'само',
]
count_vect = CountVectorizer(stop_words=stopwords)
train_counts = count_vect.fit_transform(train_data.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, train_data.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, train_data.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (16544, 27603)
[cross_val] F1: 0.526215580296729
Test counts shape: (4136, 27603)
              precision    recall  f1-score   support

    negative       0.72      0.46      0.56       535
     neutral       0.40      0.11      0.17       727
    positive       0.77      0.96      0.86      2874

    accuracy                           0.75      4136
   macro avg       0.63      0.51      0.53      4136
weighted avg       0.70      0.75      0.70      4136



# Oversampling

In [64]:
negative_df = train_data.loc[train_data['new_rating'] == 'negative']
n_neg = len(negative_df)
print('Number of negative records:', n_neg)
neutral_df = train_data.loc[train_data['new_rating'] == 'neutral'] #.sample(n=,random_state=0)
n_neu = len(neutral_df)
print('Number of neutral records:', n_neu)
positive_df = train_data.loc[train_data['new_rating'] == 'positive'] #.sample(n=n,random_state=0)
n_pos = len(positive_df)
print('Number of positive records:', n_pos)
normalized_df = pd.concat([negative_df, neutral_df, negative_df, neutral_df, positive_df, negative_df, neutral_df])
normalized_df

Number of negative records: 2141
Number of neutral records: 2908
Number of positive records: 11495


,user_id,text,rating,lemmatized_text,new_rating
0c811a8fc0497f8a6de3f254dc24e5a8,Богдан Городенський,rozetka прислали жорсткий диск в куску картонк...,1,rozetka прислати жорсткий диск в кусок картонк...,negative
e189a52c092accb7ec4d157ef72e8d57,Юра,Ноутбук не загружається з цією оперативкою. Ск...,1,ноутбук не загружаватися з цей оперативка . ск...,negative
484885177a0825a7ad30791461fefd52,Сергей Олыва,"Загалом нормальний дешевий роутер, але сигнал ...",3,"загал нормальний дешевий роутерти , але сигнал...",negative
77bde83134cf22bf5c7e308ed2910e4a,Грицько Тара,Абсолютно не рекомендую Недоліки: якість жахли...,1,абсолютно не рекомендувати недолік : якість жа...,negative
b746cfae94bcaa22d35e4bb42f83dc32,Елена Алиманска,"на роботі така сама мишка і працює відмінно,мя...",1,на робот такий самий мишка і працювати відмінн...,negative
...,...,...,...,...,...
6a8929b43ed7960ebe9b5a0c2e1070eb,Лыженко Любовь,купил не давно доволен тих в работе можно за т...,4,купил не давно доволен той в работ можно за та...,neutral
ebed3ca9d826094d98cbb7e07caf4138,Misha Petrov,Тиждень в користуванні. Перших 2 дні я кайфува...,4,тиждень в користування . перший 2 дно я кайфув...,neutral
e3ac736d45eef6d295403ed0a7290a7f,Артем Де,Брав малій на подарунок на 11-річчя. Вона задо...,4,брати малий на подарунок на 11-річчя . вона за...,neutral
0375e8174cbd45b2e1df6890fae20a44,Yura Chamber Beats,дуже гарно все Переваги: круто Недоліки: немає,4,дуже гарно весь перевага : круто недолік : немає,neutral


In [66]:
count_vect = CountVectorizer(stop_words=stopwords)
train_counts = count_vect.fit_transform(normalized_df.lemmatized_text)
print('Train counts shape:', train_counts.shape)

clf = MultinomialNB()
scores = cross_val_score(clf, train_counts, normalized_df.new_rating.to_list(), cv=5, scoring='f1_macro')
print('[cross_val] F1:', sum(scores)/5)

clf.fit(train_counts, normalized_df.new_rating.to_list())

test_counts = count_vect.transform(test_data.lemmatized_text)
print('Test counts shape:', test_counts.shape)
y_pred = clf.predict(test_counts)
test_data['predicted'] = y_pred

print(classification_report(test_data.new_rating.to_list(), y_pred))

Train counts shape: (26642, 27603)
[cross_val] F1: 0.7551692699208357
Test counts shape: (4136, 27603)
              precision    recall  f1-score   support

    negative       0.60      0.59      0.59       535
     neutral       0.33      0.34      0.33       727
    positive       0.83      0.83      0.83      2874

    accuracy                           0.71      4136
   macro avg       0.59      0.58      0.59      4136
weighted avg       0.71      0.71      0.71      4136



# Думки

1. Мені здається не всі слова треба включати в підрахунок, наприклад, навіщо нам знати чи зустрічається в тексті слово 'rayzen' та багато инших.
2. Можна було підключити ще тональності і спробувати з ними, тільки мені це вже здається варто рахувати і додавати як окрему feature, так само і ваги класів, що генерує NB. Потім підрахувати ще якісь инші властивості, і вже це використати, наприклад, для бустінгу.

Взагалі, хотілося б трохи більше зануритися в тему, щоб почати більше розуміти 
![more_practice](./more_practice.png)